# Federated Learning Project
This notebook demonstrates how to set up and compare Federated Learning (FL) with Centralized Learning (CL) using the CIFAR-100 dataset and the modified version of the LeNet-5 model taken from [Hsu et al., Federated Visual Classification with Real-World Data Distribution, ECCV 2020].

## 1. Setup
We start by importing necessary libraries and setting global constants for the experiments.

In [1]:
import sys
import torch
import torch.nn as nn

from models.model import LeNet5 #import the model

sys.path.append('../data/cifar100/')
from cifar100_loader import load_cifar100

from utils.federated_utils import fedAvg,plot_client_selection

# Constants

In [2]:
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
NUM_CLIENTS = 100  # Total number of clients in the federation
FRACTION_CLIENTS = 0.1  # Fraction of clients selected per round (C)
LOCAL_EPOCHS = 4  # Number of local steps (J)
GLOBAL_ROUNDS = 2000  # Total number of communication rounds

BATCH_SIZE = 32  # Batch size for local training
LR = 0.01  # Initial learning rate for local optimizers: best one from the centralized one
MOMENTUM = 0.9  # Momentum for SGD optimizer
WEIGHT_DECAY = 0.0001  # Regularization term for local training

LOG_FREQUENCY = 10  # Frequency of logging training progress

## 2. Data Loading
We load the CIFAR-100 dataset and split it into training, validation, and test sets. This is done using the `data_loader.py` module.

In [3]:
#load the dataset
trainloader, validloader, testloader = load_cifar100(batch_size=BATCH_SIZE, validation_split=0.25)

Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified


## 3. Federated Training
We simulate federated learning by splitting the dataset into shards and training with selected clients in each round.

### Initialize Model & Loss

In [4]:
global_model = LeNet5()
criterion = nn.NLLLoss()# our loss function for classification tasks on CIFAR-100

# Hyperparameter tuning for the first federated training baseline

In [5]:
lr = [0.05, 0.01, 0.005, 0.0001]
wd = [0.001, 0.0005, 0.0001]
rounds = 200 #fewer communication rounds for hyperparameter tuning
results = []
best_val_accuracy = 0
best_setting = None
for l in lr:
    for w in wd:
        print(f"Learning rate: {l}, Weight decay: {w}")
        global_model = LeNet5()
        #global_model,dataset, valid_dataset, num_clients,num_classes, rounds,lr,wd, C=0.1, local_steps=4,gamma=None
        val_accuracies,val_losses,train_accuracies,train_losses,global_model,client_selection_count = fedAvg(global_model, trainloader, validloader, NUM_CLIENTS, 100, rounds, l, w)
        print(f"Validation accuracy: {val_accuracies[-1]} with lr: {l} and wd: {w}")
        if val_accuracies[-1] > best_val_accuracy:
            best_val_accuracy = val_accuracies[-1]
            best_setting = (l,w)
        results.append({
                'learning_rate': l,
                'weight_decay': w,
                'train_accuracies': train_accuracies,
                'train_losses': train_losses,
                'val_accuracies': val_accuracies,
                'val_losses': val_losses,
                'client_selection_count': client_selection_count
        })
print(f"Best setting: {best_setting} with validation accuracy: {best_val_accuracy}")

Learning rate: 0.05, Weight decay: 0.001


KeyboardInterrupt: 